# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [6]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [8]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [10]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!


### ANSWER



1. Hierarchical Navigable Small World (HNSW) - which is used to implement Approximate Nearest Neighbors. Qdrant builds a layered graph and uses smart search methods to quickly and accurately find similar vectors. The flexible and parallel processing abilities of HNSW boost Qdrant's overall performance even further.

2. Product Quantization- It works by splitting high-dimensional vectors into vectors that are less precise. This redcues requried storage and it allows faster computation.

3. Qdrant can use one of two options for storage, In-memory storage (Stores all vectors in RAM, has the highest speed since disk access is required only for persistence), or Memmap storage, (creates a virtual address space associated with the file on disk).



#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [11]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [13]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '7631aeb3f4ee407885d86fca9da2e2d8', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [14]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [15]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [17]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.


### ANSWER


Firsty, asked question gets sent to pipeline, then we look at out qdrant vectorstore for most similiar document. THen the question and the retrieved context are then fed into a prompt template. This formatted prompt is sent to the "primary_qa_llm", which generates a response.




Let's test it out!

In [18]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of it.


In [19]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': 'c73bf909879147f4814995670c4eebd5', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [20]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

### ANSWER

Bigger chunk_size ensure more variations in data, and by having the same size in both cases we risk not having diverse enough questions that will test our RAG properly (like in production). Larger chunks can capture more context, which can be beneficial for understanding and generating responses that require a broader context.


In [21]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.run_config import RunConfig

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo") <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

run_config = RunConfig(
    timeout=60,
    max_retries=10,
    max_wait = 180,
    max_workers= 1
)

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings,
    run_config = run_config 
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False,raise_exceptions=True)
testset.to_pandas()

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

### ANSWER
It refers to the weighting of different types of evaluation metrics or question types used to assess the performance:
Here are the explainations for them:

Simple : This likely represents straightforward or basic questions that the RAG system is expected to answer <br>
Reasoning: Rewrite the question in a way that enhances the need for reasoning to answer it effectively.<br>
Conditioning: Modify the question to introduce a conditional element, which adds complexity to the question.<br>
Multi-Context: Rephrase the question in a manner that necessitates information from multiple related sections or chunks to formulate an answer.

Let's look at the output and see what we can learn about it!

In [23]:
testset.test_data[0]

DataRow(question='How can engaging in conversation through blogging help an entrepreneur in their search for funding?', contexts=['looking for funding to blog — about their startup, about inter-\nesting things going on, about their point of view. This puts an\nentrepreneur in the Yow of conversation, which can lead to\ninteraction with VCs through the normal medium of blogging.\nAnd, when a VC does decide to take a look at you and your com-\npany, she can read your blog to get a sense of who you are and\n22\nThe Pmarca Blog Archives'], ground_truth='Engaging in conversation through blogging can help an entrepreneur in their search for funding by putting them in the flow of conversation and potentially leading to interaction with venture capitalists (VCs). When a VC decides to take a look at the entrepreneur and their company, they can read the blog to get a sense of who the entrepreneur is.', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [24]:
test_df = testset.to_pandas()

In [25]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How can engaging in conversation through blogg...,[looking for funding to blog — about their sta...,Engaging in conversation through blogging can ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How can structured procrastination be used to ...,[like?\nStructured procrastination\nThis is a ...,You should use structured procrastination to y...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,How does the tendency for quick elimination of...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,The tendency for quick elimination of doubt in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,Who is John Perry and what is his concept of S...,[like?\nStructured procrastination\nThis is a ...,John Perry is a philosophy professor at Stanfo...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,Are the anticipated career peaks for creators ...,[These three components are conspicuously link...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,Why is it important to develop concrete skills...,"[studying whatever you love the most, whether ...","If you intend to have an impact on the world, ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How does the Liking/Loving Tendency affect the...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency affects the behavio...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,How important is it for a startup to focus on ...,"[developing a large market, as opposed to Xght...",It is therefore much more important for a star...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How does the emotional rollercoaster impact te...,"[(In case you were wondering, by the way, the ...",The emotional rollercoaster can wreak havoc on...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,How does the Inconsistency-Avoidance Tendency ...,[Five: Inconsistency-Avoidance Tendency\n[Peop...,The Inconsistency-Avoidance Tendency affects p...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [26]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [27]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [28]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [29]:
response_dataset[0]

{'question': 'How can engaging in conversation through blogging help an entrepreneur in their search for funding?',
 'answer': "Engaging in conversation through blogging can help an entrepreneur in their search for funding by putting them in the flow of conversation, which can lead to interactions with venture capitalists (VCs) through the normal medium of blogging. Additionally, VCs who blog may encourage readers to communicate with them in various ways, including soliciting email pitches in certain startup categories of interest to them. Furthermore, by reading an entrepreneur's blog, a VC can get a sense of who the entrepreneur is and their company, which can be beneficial when the VC decides to take a look at the entrepreneur and their company.",
 'contexts': ['esting things going on, about their point of view. This puts an\nentrepreneur in the Yow of conversation, which can lead to\ninteraction with VCs through the normal medium of blogging.',
  'At best, a VC blogger may encourag

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [30]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

results = evaluate(response_dataset, metrics,run_config=run_config)

In [32]:
results

{'faithfulness': 0.6485, 'answer_relevancy': 0.8435, 'context_recall': 0.7281, 'context_precision': 0.8275, 'answer_correctness': 0.4801}

In [33]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How can engaging in conversation through blogg...,Engaging in conversation through blogging can ...,"[esting things going on, about their point of ...",Engaging in conversation through blogging can ...,0.857143,1.000000,1.000000,1.000000,0.622478
1,How can structured procrastination be used to ...,Structured procrastination can be used to acco...,[standing.)\nThe gist of Structured Procrastin...,You should use structured procrastination to y...,1.000000,0.969390,1.000000,1.000000,0.482758
2,How does the tendency for quick elimination of...,The tendency for quick elimination of doubt in...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,The tendency for quick elimination of doubt in...,0.666667,0.998119,1.000000,0.916667,0.655052
3,Who is John Perry and what is his concept of S...,John Perry is the genius mind behind the conce...,"[the Apollo 11 mission, and it writes upside d...",John Perry is a philosophy professor at Stanfo...,1.000000,0.912391,1.000000,0.833333,0.544737
4,Are the anticipated career peaks for creators ...,Earlier,"[early, end late, and produce at above-average...",nan,NaN,0.000000,0.333333,1.000000,0.953437
5,Why is it important to develop concrete skills...,It is important to develop concrete skills dur...,[Which undergraduate degrees are useful in\nth...,"If you intend to have an impact on the world, ...",1.000000,1.000000,1.000000,0.916667,0.570560
6,How does the Liking/Loving Tendency affect the...,The Liking/Loving Tendency can be a major impe...,[other facts to facilitate love.\nThe applicat...,The Liking/Loving Tendency affects the behavio...,1.000000,0.933404,1.000000,1.000000,0.401900
7,How important is it for a startup to focus on ...,It is important for a startup to focus on deve...,"[competitor, be sure to take a step back and s...",It is therefore much more important for a star...,0.000000,0.934026,1.000000,1.000000,0.479481
8,How does the emotional rollercoaster impact te...,The emotional rollercoaster wreaks havoc not j...,[This combines the Xrst and second items above...,The emotional rollercoaster can wreak havoc on...,0.666667,0.990438,0.500000,1.000000,0.425165
9,How does the Inconsistency-Avoidance Tendency ...,The Inconsistency-Avoidance Tendency makes peo...,[Five: Inconsistency-Avoidance Tendency\n[Peop...,The Inconsistency-Avoidance Tendency affects p...,1.000000,0.931311,0.250000,1.000000,0.573948


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [34]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [36]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [37]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [38]:
print(response["answer"])

I'm sorry, but based on the context provided, I do not have information about any feud involving Taylor Swift.


In [39]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [40]:
print(response["answer"])

The text does not provide any information about specific individuals or groups feuding. It mainly discusses human tendencies towards dislike and hate, the history of war, and factors contributing to success in different industries.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [41]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [42]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics,run_config=run_config)

In [44]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How can engaging in conversation through blogg...,Engaging in conversation through blogging can ...,"[esting things going on, about their point of ...",Engaging in conversation through blogging can ...,0.666667,1.000000,1.000000,1.000000,0.559666
1,How can structured procrastination be used to ...,Structured procrastination can be used to acco...,[standing.)\nThe gist of Structured Procrastin...,You should use structured procrastination to y...,1.000000,0.970069,1.000000,1.000000,0.509993
2,How does the tendency for quick elimination of...,The tendency for quick elimination of doubt in...,[do…\nSo pronounced is the tendency in man to ...,The tendency for quick elimination of doubt in...,1.000000,0.885497,1.000000,0.916667,0.688993
3,Who is John Perry and what is his concept of S...,John Perry is the genius mind behind the conce...,[standing.)\nThe gist of Structured Procrastin...,John Perry is a philosophy professor at Stanfo...,1.000000,0.912391,1.000000,0.950000,0.739563
4,Are the anticipated career peaks for creators ...,The anticipated career peaks for creators who ...,"[early, end late, and produce at above-average...",nan,1.000000,0.982765,0.000000,0.805556,0.184018
5,Why is it important to develop concrete skills...,It is important to develop concrete skills dur...,[Which undergraduate degrees are useful in\nth...,"If you intend to have an impact on the world, ...",1.000000,0.914746,1.000000,0.854167,0.563017
6,How does the Liking/Loving Tendency affect the...,The Liking/Loving Tendency can act as a condit...,[other facts to facilitate love.\nThe applicat...,The Liking/Loving Tendency affects the behavio...,0.750000,0.960913,0.333333,0.770833,0.489372
7,How important is it for a startup to focus on ...,It is very important for a startup to focus on...,"[answer, in part because in the beginning of a...",It is therefore much more important for a star...,0.750000,0.980877,1.000000,1.000000,0.644380
8,How does the emotional rollercoaster impact te...,The emotional rollercoaster in a startup can l...,[This combines the Xrst and second items above...,The emotional rollercoaster can wreak havoc on...,0.800000,0.951433,0.500000,0.926667,0.333737
9,How does the Inconsistency-Avoidance Tendency ...,The Inconsistency-Avoidance Tendency makes peo...,[less brain-blocked by its previous conclusion...,The Inconsistency-Avoidance Tendency affects p...,1.000000,0.901524,0.250000,0.816667,0.617422


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [45]:
results

{'faithfulness': 0.6485, 'answer_relevancy': 0.8435, 'context_recall': 0.7281, 'context_precision': 0.8275, 'answer_correctness': 0.4801}

And see how our advanced retrieval modified our chain!

In [46]:
advanced_retrieval_results

{'faithfulness': 0.8608, 'answer_relevancy': 0.9446, 'context_recall': 0.6711, 'context_precision': 0.8135, 'answer_correctness': 0.5439}

In [47]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.648459,0.860840,0.212380
1,answer_relevancy,0.843505,0.944553,0.101048
2,context_recall,0.728070,0.671053,-0.057018
3,context_precision,0.827485,0.813520,-0.013965
4,answer_correctness,0.480108,0.543874,0.063766


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [48]:
# we change the embedding model to "text-embedding-3-small"
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [49]:
# we store our qdrant vector store in memory
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [50]:
# we create new retriver from our vector store
new_retriever = vector_store.as_retriever()

In [51]:
# we create  MultiQueryRetriever that fetches data from the vectoresore
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [52]:
# "Create retrieval chain that retrieves documents and then passes them on."
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [53]:
# we append the "answers" and "contexts" to a list for each response we got from our questons
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [54]:
# we convert items from dict to dataset
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

# we evaluate the new results
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics,run_config=run_config)

In [56]:
# we print results
new_advanced_retrieval_results

{'faithfulness': 0.8292, 'answer_relevancy': 0.9321, 'context_recall': 0.6372, 'context_precision': 0.7309, 'answer_correctness': 0.5107}

In [57]:
# we create dataset from given inputs, so we can cleary compare our models
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.648459,0.860840,0.829198,-0.031642,0.180739
1,answer_relevancy,0.843505,0.944553,0.932058,-0.012495,0.088553
2,context_recall,0.728070,0.671053,0.637218,-0.033835,-0.090852
3,context_precision,0.827485,0.813520,0.730934,-0.082587,-0.096552
4,answer_correctness,0.480108,0.543874,0.510711,-0.033164,0.030602


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

### ANSWER

I would not say that we made significant changes, but we made slight improvments.
Answer Relevancy 5% decrease
Context recall 9.5% improvement
Context precision 1.6% improvement
answer correctness  2% improvement

The question is how did it improve in real life? It is hard to tell.

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [58]:
### YOUR CODE HERE